In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
df = pd.read_csv('train.csv')
df_train = df[['SalePrice', 'OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']]

df_train.head()

,SalePrice,OverallQual,GrLivArea,GarageCars,TotalBsmtSF,FullBath,YearBuilt
0,208500,7,1710,2,856,2,2003
1,181500,6,1262,2,1262,2,1976
2,223500,7,1786,2,920,2,2001
3,140000,7,1717,3,756,1,1915
4,250000,8,2198,3,1145,2,2000


In [5]:
# 説明変数と目的変数
X, y = df_train.drop('SalePrice', 1), df_train.SalePrice

In [7]:
from sklearn.linear_model import RANSACRegressor

# RANSACの作成
ran = RANSACRegressor(min_samples=int(len(X) * 0.9),
                      random_state=0)
# フィッティング
model = ran.fit(X, y)
# パラメータ（定数項と係数）の確認
ran.estimator_.intercept_, ran.estimator_.coef_

(-706005.2405882915,
 array([16316.38178663,    51.33568501, 12602.09476156,    29.84940061,
        -4368.71146983,   334.65536743]))

In [8]:
df_test = pd.read_csv('test.csv')

df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [10]:
df_output = df_test[['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF', 'FullBath', 'YearBuilt']]

df_output.head()

,OverallQual,GrLivArea,GarageCars,TotalBsmtSF,FullBath,YearBuilt
0,5,896,1.0,882.0,1,1961
1,6,1329,1.0,1329.0,1,1958
2,5,1629,2.0,928.0,2,1997
3,6,1604,2.0,926.0,2,1998
4,8,1280,2.0,1280.0,2,1992


In [23]:
df_output.shape

(1459, 6)

In [12]:
# モデルに打ち込んだら欠損値があったので、処理する。
# 列ごとに欠損値かどうか
df_output_column = df_output.isnull().any(0)
df_output_column

OverallQual    False
GrLivArea      False
GarageCars      True
TotalBsmtSF     True
FullBath       False
YearBuilt      False
dtype: bool

In [13]:
# 欠損値をカウント
df_output_number = df_output.isnull().sum()
df_output_number

OverallQual    0
GrLivArea      0
GarageCars     1
TotalBsmtSF    1
FullBath       0
YearBuilt      0
dtype: int64

In [17]:
#1つだけなので、平均を割り当てる
from sklearn.preprocessing import Imputer

#  欠損値(NaN) に対して 平均値(mean) を割り当てるImputerオブジェクトを生成
imp = Imputer(missing_values="NaN", strategy="mean")

# データセットの学習
imp.fit(df_output[["GarageCars"]])

# strategy="mean"に基づいて欠損値(NaN)を平均値に置き換える
values = imp.transform(df_output[["GarageCars"]])

# DataFrameに変換したデータを代入する
df_output[["GarageCars"]] = values

df_output_number = df_output.isnull().sum()
df_output_number

OverallQual    0
GrLivArea      0
GarageCars     0
TotalBsmtSF    1
FullBath       0
YearBuilt      0
dtype: int64

In [18]:
#  欠損値(NaN) に対して 平均値(mean) を割り当てるImputerオブジェクトを生成
imp = Imputer(missing_values="NaN", strategy="mean")

# データセットの学習
imp.fit(df_output[["TotalBsmtSF"]])

# strategy="mean"に基づいて欠損値(NaN)を平均値に置き換える
values = imp.transform(df_output[["TotalBsmtSF"]])

# DataFrameに変換したデータを代入する
df_output[["TotalBsmtSF"]] = values

df_output_number = df_output.isnull().sum()
df_output_number

OverallQual    0
GrLivArea      0
GarageCars     0
TotalBsmtSF    0
FullBath       0
YearBuilt      0
dtype: int64

In [21]:
y_pred = model.predict(df_output)
y_pred

array([112393.17227151, 163276.62163566, 171676.2783305 , ...,
       151707.21135587, 114859.71660918, 236647.81357525])

In [30]:
df_output_pred = pd.DataFrame(y_pred, columns=['SalePrice'])
df_output_pred.head()

,SalePrice
0,112393.172272
1,163276.621636
2,171676.278330
3,186984.224558
4,211542.981798


In [31]:
df_output_id = df_test[['Id']]
df_output_id.head()

,Id
0,1461
1,1462
2,1463
3,1464
4,1465


In [32]:
output = pd.concat([df_output_id, df_output_pred], axis=1)
output

,Id,SalePrice
0,1461,112393.172272
1,1462,163276.621636
2,1463,171676.278330
3,1464,186984.224558
4,1465,211542.981798
5,1466,183063.615358
6,1467,170792.866651
7,1468,175759.196459
8,1469,202654.465544
9,1470,110972.083963


In [33]:
output.to_csv("RANSACRegressor_20180728.csv", index = False)

# 反省
4278位だった...
もっと変数の扱いを丁寧にするべきだった。

→上位ランカーの人は、ダミー変数化だったり変数の意味を考えて自分で二値の分類をしてみたりしてる。


→変数選択はしなくて良いかも。クオリティーコントロール程度に留めておくのがベターかなぁ。（多重共線性とか）

（そこまで相関強くなくても、結果に寄与する変数をカットしてしまってたんだろうな。）

└その上で、正則化して係数に重み付けする。

→多重回帰は、変数の正規性を前提としているので、その確認も足りていなかった。

    
とりあえず次は、非線形系のモデルを試す！